In [1]:
import pathlib
import os
import torch
import PIL
from PIL import Image
from torch.utils.data import Dataset
from utils import utils
from dataset.dataset import ASLDataset
from model.TwoStreamCNN import TwoStreamCNN
import torch
from torch import nn
import numpy as np
from torch.optim import Adam
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from utils import utils
from torchvision.transforms import v2
import torch 
import torch.optim as optim

In [2]:
import torchvision
print(torchvision.__version__)

0.16.0+cpu


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size=(226, 226), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
])

In [5]:
# 1. Load and transform data

train_dir = "asl_alphabet_train/asl_alphabet_train/"
test_dir = "asl_alphabet_test"
train_data_simple = ASLDataset(data_path=train_dir, transform=transform)
test_data_simple = ASLDataset(data_path=test_dir, transform=transform)

# 2. Turn data into DataLoaders
import os
from torch.utils.data import DataLoader

# Setup batch size and number of workers 
BATCH_SIZE = 1
NUM_WORKERS = os.cpu_count()
print(f"Creating DataLoader's with batch size {BATCH_SIZE} and {NUM_WORKERS} workers.")

# Create DataLoader's
train_dataloader_simple = DataLoader(train_data_simple, 
                                     batch_size=BATCH_SIZE, 
                                     shuffle=True, 
                                     num_workers=NUM_WORKERS)

test_dataloader_simple = DataLoader(test_data_simple, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False, 
                                    num_workers=NUM_WORKERS)

train_dataloader_simple, test_dataloader_simple

Creating DataLoader's with batch size 1 and 8 workers.


(<torch.utils.data.dataloader.DataLoader at 0x7f11bc9abc50>,
 <torch.utils.data.dataloader.DataLoader at 0x7f11bdf6e5d0>)

In [22]:
len(train_dataloader_simple)

87000

In [23]:
len(test_dataloader_simple)

28

In [21]:
len(next(iter(train_dataloader_simple)))

3

In [10]:
for _, image in enumerate(train_dataloader_simple):
    print(len(image))
    break

3


In [17]:
next(iter(test_data_simple))[1]

Image([[[0.6235, 0.6235, 0.6275,  ..., 0.6667, 0.6667, 0.6667],
        [0.6235, 0.6235, 0.6314,  ..., 0.6667, 0.6667, 0.6667],
        [0.6275, 0.6275, 0.6314,  ..., 0.6667, 0.6627, 0.6627],
        ...,
        [0.3529, 0.3529, 0.3529,  ..., 0.4627, 0.4627, 0.4627],
        [0.3490, 0.3490, 0.3490,  ..., 0.4627, 0.4627, 0.4627],
        [0.3490, 0.3490, 0.3490,  ..., 0.4627, 0.4627, 0.4627]],

       [[0.7333, 0.7333, 0.7333,  ..., 0.7451, 0.7451, 0.7451],
        [0.7333, 0.7333, 0.7333,  ..., 0.7451, 0.7451, 0.7451],
        [0.7333, 0.7333, 0.7333,  ..., 0.7451, 0.7412, 0.7412],
        ...,
        [0.2627, 0.2627, 0.2627,  ..., 0.4667, 0.4667, 0.4667],
        [0.2588, 0.2588, 0.2627,  ..., 0.4667, 0.4667, 0.4667],
        [0.2588, 0.2588, 0.2588,  ..., 0.4667, 0.4667, 0.4667]],

       [[0.7490, 0.7490, 0.7529,  ..., 0.7725, 0.7725, 0.7725],
        [0.7490, 0.7490, 0.7529,  ..., 0.7725, 0.7725, 0.7725],
        [0.7490, 0.7490, 0.7529,  ..., 0.7725, 0.7686, 0.7686],
        ..

In [20]:
import matplotlib.pyplot as plt
def plot_image(image):
    plt.figure(figsize=(10, 10))
    plt.imshow(image.permute(1, 2, 0))

plot_image(next(iter(train_data_simple))[1])

AttributeError: 'int' object has no attribute 'permute'

<Figure size 1000x1000 with 0 Axes>

In [6]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X,XN, y) in enumerate(dataloader):
        # Send data to target device
        X,XN, y = X.to(device),XN.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X,XN)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [7]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval() 
    
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
    
    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X,XN, y) in enumerate(dataloader):
            # Send data to target device
            X,XN, y = X.to(device),XN.to(device), y.to(device)
    
            # 1. Forward pass
            test_pred_logits = model(X,XN)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            
            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [8]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

c:\Users\hp\anaconda3\envs\twostream\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 10

# Recreate an instance of TinyVGG
model_0 = TwoStreamCNN(29)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# Check the model_0_results keys
model_0_results.keys()

In [ ]:
def plot_loss_curves(results: Dict[str, List[float]]):
    """Plots training curves of a results dictionary.

    Args:
        results (dict): dictionary containing list of values, e.g.
            {"train_loss": [...],
             "train_acc": [...],
             "test_loss": [...],
             "test_acc": [...]}
    """
    
    # Get the loss values of the results dictionary (training and test)
    loss = results['train_loss']
    test_loss = results['test_loss']

    # Get the accuracy values of the results dictionary (training and test)
    accuracy = results['train_acc']
    test_accuracy = results['test_acc']

    # Figure out how many epochs there were
    epochs = range(len(results['train_loss']))

    # Setup a plot 
    plt.figure(figsize=(15, 7))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='train_loss')
    plt.plot(epochs, test_loss, label='test_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='train_accuracy')
    plt.plot(epochs, test_accuracy, label='test_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [ ]:
plot_loss_curves(model_0_results)